In [ ]:
import os
import argparse

In [ ]:
from datetime import datetime

In [ ]:
"trainer/type"

In [ ]:

! python3 -m trainer.task \
    --train_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_train.tfrecord' \
    --valid_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_valid.tfrecord' \
    --test_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_test.tfrecord' \
    --output_path='/home/jupyter/output/model/test_training/'\
    --neurons=10 \
    --batch_size=32 

In [ ]:
python3 task.py \
    --train_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_train.tfrecord' \
    --valid_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_valid.tfrecord' \
    --test_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_test.tfrecord' \
    --output_path='/home/jupyter/output/model/test_training/'\
    --neurons=10 \
    --batch_size=32 

### Train Locally Using gcloud

In [ ]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
! gcloud config set ml_engine/local_python $(which python3)

In [ ]:
# This is similar to `python -m trainer.task --job-dir local-training-output`
# but it better replicates the AI Platform environment, especially for
# distributed training (not applicable here).
! gcloud ai-platform local train \
    --package-path="./trainer/" \
    --module-name="trainer.task_all_events-LOS" \
    -- \
    --train_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_train.tfrecord' \
    --valid_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_valid.tfrecord' \
    --test_input='/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/all_events_test.tfrecord' \
    --output_path='/home/jupyter/output/'\
    --num_dense_layers=3 \
    --neurons=128 \
    --batch_size=128 \
    --learning_rate=0.001 \
    --num_epochs=25 \
    --dropout=0 \
    --embedding_dropout=0.5\
    --embed_dim=15\
    --hypertune='nothypertune'
    
    

### Train on GCP AI-PLATFORM Using gcloud

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%Y%m%d_%H%M%S")
date_time

In [ ]:
job_name = "#Training Job Name"+date_time
job_name

In [ ]:
import os

In [ ]:
os.environ["job_name"] = job_name
os.environ['data_path'] = "gs://"#path to training storage bucket with training data 

In [ ]:
%%bash
echo $job_name
echo $data_path

In [ ]:
%%bash
echo 'gs://test_training/logs/'$job_name

In [ ]:
%%bash
gcloud beta ai-platform jobs submit training $job_name \
    --staging-bucket="gs://test_training/" \
    --package-path="./trainer/" \
    --module-name="trainer.task_all_events-LOS" \
    --region=#region to submit job \
    --scale-tier="CUSTOM" \
    --master-machine-type="standard_gpu" \
    --python-version="3.5" \
    --runtime-version="1.13" \
    --config="hptuning_config.yaml"\
    -- \
    --train_input=$data_path'all_events_train.tfrecord' \
    --valid_input=$data_path'all_events_valid.tfrecord' \
    --test_input=$data_path'all_events_test.tfrecord' \
    --output_path='gs://test_training/'$job_name \
    --num_dense_layers=3 \
    --neurons=128 \
    --batch_size=128 \
    --learning_rate=0.001 \
    --num_epochs=20 \
    --dropout=0 \
    --embedding_dropout=0.5\
    --embed_dim=15\
    --hypertune='hypertune'